# Grammatical Errors Dataset
- Do the LMs express similar trends for these datasets?

In [ ]:
import os, sys, time
sys.path.append("../")

import src.anomaly_model

import numpy as np 
import torch
import transformers 
import pprint 
pprint = pprint.PrettyPrinter()

from utils import timed_func

In [14]:
@timed_func
def parse_grammar_dataset():
    m2_path = "../data/BEA14/WI_LOCNESS/wi+locness/m2/A.train.gold.bea19.m2"
    with open(m2_path, "r") as f:
        raw_lines = f.readlines()

    data = []
    curr_item = {}
    for line in raw_lines:
        if line.startswith("S "):
            if len(curr_item) > 0 and len(curr_item["errors"]) > 0:
                data.append(curr_item)
            curr_item = {"sentence": line[2:], "errors": []}
        elif line.startswith("A "):
            items = line.split("|||")
            start_pos_str, end_pos_str = items[0][2:].split()
            start_pos, end_pos = int(start_pos_str), int(end_pos_str)
            error = {
                "start_pos": start_pos,  # These are in terms of word locs
                "end_pos": end_pos,
                "correct": items[2] 
            }
            curr_item["errors"].append(error)

    print("Collected {} sentences".format(len(data)))
    sentences = [item['sentence'] for item in data]
    return data, sentences 

gec_data, sentences = parse_grammar_dataset()
    
        

Collected 10492 sentences
parse_grammar_dataset done in 0.82 seconds.


```
This is an example of what an M2 file looks like
S It 's difficult answer at the question " what are you going to do in the future ? " if the only one who has to know it is in two minds .
A 3 3|||M:VERB:FORM|||to|||REQUIRED|||-NONE-|||0
A 4 5|||U:PREP||||||REQUIRED|||-NONE-|||0
```


In [11]:
line = "A 6 6|||M:PUNCT|||,|||REQUIRED|||-NONE-|||0"
line.split("|||")

['A 6 6', 'M:PUNCT', ',', 'REQUIRED', '-NONE-', '0']

In [15]:
model = src.anomaly_model.AnomalyModel(sentences)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [ ]:
def all_layer_scores(sent):
  tokens, all_layer = model.gmm_score([sent])
  tokens = tokens[0]
  all_layer = all_layer[0]
  plt.figure(figsize=(8, 8))
  plt.imshow(all_layer, origin='lower')
  plt.xticks(range(len(tokens)), tokens, rotation='vertical')
  plt.yticks(range(12), range(12))
  plt.ylabel('Layer')
  plt.show()

pprint(gec_data[0])
all_layer_scores(sentences[0])